In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
import requests
import os
from dotenv import load_dotenv

load_dotenv()
MISTRAL_API_KEY = os.getenv(os.getenv("MISTRAL_API_KEY"))  # Store your key as env variable
MISTRAL_API_URL = "https://api.mistral.ai/v1/chat/completions"
MISTRAL_MODEL = "mistral-small"  # or "mistral-medium", etc.

def mistral_call(prompt: str) -> str:
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MISTRAL_MODEL,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post(MISTRAL_API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Mistral API error: {response.text}")

# Wrap it as a LangChain Runnable
mistral_model = RunnableLambda(lambda x: mistral_call(x["question"]))

# Build prompt
template = "Question: {question}"
prompt = ChatPromptTemplate.from_template(template)

# Chain = prompt → mistral_model
chain = prompt | mistral_model
